Level Sets and Gradients Jupyter Notebook

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, clear_output

#### **The cell below is used to see the 2D level sets for certain 3D Visualizations.**

There are three visualizations that we will focus on in these visualizations: 
**Sin/Cosine, Monkey Saddle, and Paraboloid** (don't worry about the names for now)

In [3]:
def f_original(x, y):
    return 0.5 * np.sin(x) * np.cos(y) + 0.15 * (x**2 - y**2)

# Monkey saddle: threefold saddle with a flat critical point at origin
# Scaled for a comparable z-range on [-3, 3]^2

def f_monkey_saddle(x, y):
    return 0.06 * (x**3 - 3.0 * x * y**2)

# Elliptic paraboloid: smooth convex bowl, simple gradients

def f_paraboloid(x, y):
    return 0.12 * (x**2 + y**2)

In [4]:
surface_funcs_2d = {
    "Original (sin/cos + saddle)": f_original,
    "Monkey saddle": f_monkey_saddle,
    "Paraboloid": f_paraboloid,
}
x2 = np.linspace(-3.0, 3.0, 240)
y2 = np.linspace(-3.0, 3.0, 240)
X2, Y2 = np.meshgrid(x2, y2)
_default_key_2d = "Original (sin/cos + saddle)"
Z2 = surface_funcs_2d[_default_key_2d](X2, Y2)
z2min, z2max = float(Z2.min()), float(Z2.max())
def compute_level_set_polylines_2d(level: float, Z_arr: np.ndarray) -> list[np.ndarray]:
    fig, ax = plt.subplots()
    cs = ax.contour(x2, y2, Z_arr, levels=[level])
    paths = []
    try:
        if hasattr(cs, "allsegs") and cs.allsegs and len(cs.allsegs[0]) > 0:
            for seg in cs.allsegs[0]:
                v = np.asarray(seg)
                if v.shape[0] > 1:
                    paths.append(v)
        elif hasattr(cs, "collections") and cs.collections:
            for p in cs.collections[0].get_paths():
                v = p.vertices
                if v.shape[0] > 1:
                    paths.append(v)
    finally:
        plt.close(fig)
    return paths
ls_dropdown = widgets.Dropdown(
    options=list(surface_funcs_2d.keys()),
    value=_default_key_2d,
    description="Function",
    layout=widgets.Layout(width="320px"),
)
num_levels = 10
levels_rel = np.linspace(0.05, 0.95, num_levels)
out2d_levelsets = widgets.Output()
_is_rendering_2d = False
def _update_Z2_for_current():
    global Z2, z2min, z2max
    f = surface_funcs_2d[ls_dropdown.value]
    Z2 = f(X2, Y2)
    z2min, z2max = float(Z2.min()), float(Z2.max())
def render_levelset_2d():
    global _is_rendering_2d
    if _is_rendering_2d:
        return
    _is_rendering_2d = True
    try:
        _update_Z2_for_current()
        if z2max == z2min:
            selected_levels = [z2min]
        else:
            selected_levels = list(z2min + levels_rel * (z2max - z2min))
        fig = go.Figure()
        reds = px.colors.sequential.Reds
        n_palette = len(reds)
        for idx, lvl in enumerate(selected_levels):
            color_idx = int(round(idx * (n_palette - 1) / max(1, len(selected_levels) - 1)))
            color = reds[color_idx]
            paths = compute_level_set_polylines_2d(lvl, Z2)
            first_for_level = True
            for verts in paths:
                fig.add_trace(
                    go.Scatter(x=verts[:, 0],y=verts[:, 1],mode="lines",line=dict(color=color, width=2.8),name=f"z = {lvl:.2f}",showlegend=first_for_level,
                    )
                )
                first_for_level = False
        fig.update_layout(xaxis_title="x",yaxis_title="y",yaxis=dict(scaleanchor="x", scaleratio=1),legend=dict(x=1.02, xanchor="left", y=1.0, yanchor="top", title=dict(text="Level (z)"), bgcolor="rgba(255,255,255,0.7)"),margin=dict(l=0, r=160, t=28, b=0),title=f"2D Level Sets — {ls_dropdown.value}",
        )
        with out2d_levelsets:
            clear_output(wait=True)
            display(fig)
    finally:
        _is_rendering_2d = False
ls_dropdown.observe(lambda change: render_levelset_2d(), names="value")
controls_2d = widgets.HBox([
    ls_dropdown,
])
ui_2d = widgets.VBox([
    widgets.HTML("<b>2D Level Sets</b> — Multiple contours for the selected function."),
    controls_2d,
    out2d_levelsets,
])

In [ ]:
surface_funcs = {
    "Original (sin/cos + saddle)": f_original,
    "Monkey saddle": f_monkey_saddle,
    "Paraboloid": f_paraboloid,
}
x = np.linspace(-3.0, 3.0, 160)
y = np.linspace(-3.0, 3.0, 160)
X, Y = np.meshgrid(x, y)
_default_key = "Original (sin/cos + saddle)"
Z = surface_funcs[_default_key](X, Y)
zmin, zmax = float(Z.min()), float(Z.max())
def compute_level_set_polylines(level):
    fig, ax = plt.subplots()
    cs = ax.contour(x, y, Z, levels=[level])
    paths = []
    try:
        if hasattr(cs, "allsegs") and cs.allsegs and len(cs.allsegs[0]) > 0:
            for seg in cs.allsegs[0]:
                v = np.asarray(seg)
                if v.shape[0] > 1:
                    paths.append(v)
        elif hasattr(cs, "collections") and cs.collections:
            for p in cs.collections[0].get_paths():
                v = p.vertices  # shape (N, 2) with columns [x, y]
                if v.shape[0] > 1:
                    paths.append(v)
    finally:
        plt.close(fig)
    return paths
def get_current_f():
    return surface_funcs[surface_dropdown.value]
def partial_derivatives(x0: float, y0: float, h: float = 1e-3) -> tuple[float, float, float]:
    f = get_current_f()
    z0 = float(f(x0, y0))
    fx = float((f(x0 + h, y0) - f(x0 - h, y0)) / (2.0 * h))
    fy = float((f(x0, y0 + h) - f(x0, y0 - h)) / (2.0 * h))
    return z0, fx, fy
def add_tangent_traces(fig: go.Figure, x0: float, y0: float, half_len: float = 0.8, npts: int = 60) -> None:
    z0, fx, fy = partial_derivatives(x0, y0)
    xs = np.linspace(max(float(x.min()), x0 - half_len), min(float(x.max()), x0 + half_len), npts)
    ys = np.linspace(max(float(y.min()), y0 - half_len), min(float(y.max()), y0 + half_len), npts)
    z_tan_x = z0 + fx * (xs - x0)
    z_tan_y = z0 + fy * (ys - y0)
    fig.add_trace(
        go.Scatter3d(x=[x0], y=[y0], z=[z0],mode="markers",marker=dict(size=5, color="#111111"),name="Point (x0, y0, f)"
        )
    )
    fig.add_trace(
        go.Scatter3d(x=xs,y=np.full_like(xs, y0),z=z_tan_x,mode="lines",line=dict(color="#1f77b4", width=6, dash="dash"),name="dz/dx",showlegend=True,
        )
    )
    fig.add_trace(
        go.Scatter3d(
            x=np.full_like(ys, x0),y=ys,z=z_tan_y,mode="lines",line=dict(color="#ff7f0e", width=6, dash="dash"),name="dz/dy",showlegend=True,
        )
    )
def add_tangent_plane(fig: go.Figure, x0: float, y0: float, half_size: float = 0.8, resolution: int = 24, opacity: float = 0.4) -> None:
    z0, fx, fy = partial_derivatives(x0, y0)
    xp = np.linspace(max(float(x.min()), x0 - half_size), min(float(x.max()), x0 + half_size), resolution)
    yp = np.linspace(max(float(y.min()), y0 - half_size), min(float(y.max()), y0 + half_size), resolution)
    XP, YP = np.meshgrid(xp, yp)
    ZP = z0 + fx * (XP - x0) + fy * (YP - y0)
    fig.add_trace(
        go.Surface(x=XP,y=YP,z=ZP,colorscale=[[0, "#8a2be2"], [1, "#8a2be2"]],showscale=False,opacity=opacity,name="Tangent plane",showlegend=False,
        )
    )
def add_normal_line(fig: go.Figure, x0: float, y0: float, length: float = 1.5) -> None:
    z0, fx, fy = partial_derivatives(x0, y0)
    v = np.array([fx, fy, -1.0])
    nrm = float(np.linalg.norm(v))
    if nrm == 0.0:
        nrm = 1.0
    v = v / nrm
    p1 = np.array([x0, y0, z0]) - 0.5 * length * v
    p2 = np.array([x0, y0, z0]) + 0.5 * length * v
    fig.add_trace(
        go.Scatter3d(x=[p1[0], p2[0]],y=[p1[1], p2[1]],z=[p1[2], p2[2]],mode="lines",line=dict(color="#2ca02c", width=6), name="Normal line",showlegend=True,
        )
    )
def build_3d_figure(z0: float, show_plane: bool, birds_eye: bool) -> go.Figure:
    fig = go.Figure()
    fig.add_trace(
        go.Surface(x=X,y=Y,z=Z,colorscale="Viridis",reversescale=False,showscale=False,colorbar=dict(title="Height"),name="Surface",opacity=0.55,
        )
    )
    if show_plane:
        plane_z = np.full_like(Z, z0)
        fig.add_trace(
            go.Surface(x=X,y=Y,z=plane_z,colorscale=[[0, "#AAAAAA"], [1, "#AAAAAA"]],showscale=False,opacity=0.30,name=f"Plane z={z0:.2f}",
            )
        )
    level_paths = compute_level_set_polylines(z0)
    for verts in level_paths:
        fig.add_trace(
            go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=np.full(verts.shape[0], z0),mode="lines",line=dict(color="#FF4136", width=6),name=f"Contour at z={z0:.2f}",showlegend=False,
            )
        )
    scene = dict(xaxis_title="x",yaxis_title="y",zaxis_title="z",xaxis=dict(showspikes=False),yaxis=dict(showspikes=False),zaxis=dict(showspikes=False),aspectmode="data",
    )

    if birds_eye:
        fig.update_layout(scene=scene,scene_camera=dict(eye=dict(x=0.0001, y=0.0001, z=2.5),projection=dict(type="orthographic"),),margin=dict(l=0, r=0, t=0, b=0),title=f"3D View (Bird's-eye camera) — z-plane: {z0:.2f}",
        )
    else:
        fig.update_layout(scene=scene,scene_camera=dict(eye=dict(x=1.35, y=1.35, z=0.95)),margin=dict(l=0, r=0, t=100, b=0),title=f"3D Visual Representation of the Gradient",
        )

    return fig
instructions = widgets.HTML(
    value=(
        "<b>How to use:</b>"
        "<ul>"
        "<li>Select a surface from the dropdown to switch functions.</li>"
        "<li>Rotate/zoom the 3D surface. It's colored by height, with the intersection contour highlighted in red.</li>"
        "<li>Enter a point (x0, y0) and press Enter to add tangent lines, the tangent plane, and the normal line at that point.</li>"
        "</ul>"
    )
)
surface_dropdown = widgets.Dropdown(options=list(surface_funcs.keys()),value=_default_key,description="Surface",layout=widgets.Layout(width="280px"),
)
z_slider = widgets.FloatSlider(
    description="Plane z",min=zmin,max=zmax,step=(zmax - zmin) / 200.0 if zmax > zmin else 0.01,value=(zmin + zmax) / 2.0,continuous_update=False,readout_format=".2f",layout=widgets.Layout(width="350px"),
)
show_plane_chk = widgets.Checkbox(value=True, description="Show plane + slider")
birds_eye_toggle = widgets.ToggleButton(
    value=False, description="Bird’s-eye 2D view", icon="eye"
)
show_heatmap_chk = widgets.Checkbox(value=True, description="Show heatmap (2D)")
x0_input = widgets.FloatText(description="x0", value=0.5, step=0.05, layout=widgets.Layout(width="180px"))
y0_input = widgets.FloatText(description="y0", value=0.5, step=0.05, layout=widgets.Layout(width="180px"))
show_tangent_plane_chk = widgets.Checkbox(value=False, description="Show tangent plane")
out3d = widgets.Output()
out2d = widgets.Output()
current_fig3d = None
current_fig2d = None
def _update_z_stats_for_current_surface():
    global Z, zmin, zmax
    f = get_current_f()
    Z = f(X, Y)
    zmin, zmax = float(Z.min()), float(Z.max())
    z_slider.min = zmin
    z_slider.max = zmax
    z_slider.step = (zmax - zmin) / 200.0 if zmax > zmin else 0.01
    if z_slider.value < zmin or z_slider.value > zmax:
        z_slider.value = (zmin + zmax) / 2.0
def render_all():
    global current_fig3d, current_fig2d
    _update_z_stats_for_current_surface()
    z_slider.layout.display = "flex" if show_plane_chk.value else "none"
    current_fig3d = build_3d_figure(
        z0=z_slider.value, show_plane=show_plane_chk.value, birds_eye=birds_eye_toggle.value
    )
    try:
        x0v = float(x0_input.value)
        y0v = float(y0_input.value)
        if np.isfinite(x0v) and np.isfinite(y0v):
            if show_tangent_plane_chk.value:
                add_tangent_plane(current_fig3d, x0v, y0v, half_size=0.9, resolution=28, opacity=0.35)
            add_normal_line(current_fig3d, x0v, y0v, length=1.6)
            add_tangent_traces(current_fig3d, x0v, y0v, half_len=0.9)
    except Exception:
        pass
    with out3d:
        clear_output(wait=True)
        display(current_fig3d)
surface_dropdown.observe(lambda change: render_all(), names="value")
z_slider.observe(lambda change: render_all(), names="value")
show_plane_chk.observe(lambda change: render_all(), names="value")
birds_eye_toggle.observe(lambda change: render_all(), names="value")
show_heatmap_chk.observe(lambda change: render_all(), names="value")
x0_input.observe(lambda change: render_all(), names="value")
y0_input.observe(lambda change: render_all(), names="value")
show_tangent_plane_chk.observe(lambda change: render_all(), names="value")
controls_row1 = widgets.HBox([
    surface_dropdown,
])
point_row = widgets.HBox([widgets.HTML("<b>Point (press Enter):</b>&nbsp;"),x0_input,y0_input,show_tangent_plane_chk,
])

ui = widgets.VBox([instructions,controls_row1,point_row,out3d,out2d,
])


#### **The cell below is used to see the 2D level sets for certain 3D Visualizations.**

There are three visualizations that we will focus on in these visualizations: 
**Sin/Cosine, Monkey Saddle, and Paraboloid** 

(don't worry about the names for now)

In [12]:
render_levelset_2d()
display(ui_2d)

### This is a 3D visualization of the level sets shown above

To understand what the tangent plane is, click the "Tangent Plane" Checkbox below. Then, change X and Y values in order to move your gradient into the desired location!

In [13]:
render_all()
display(ui)